In [85]:
# Importação de bibliotecas
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import f1_score

# Leitura de dados

In [26]:
# Ler arquivo .csv para um DataFrame pandas
tb_hotel = pd.read_csv("tb_hotel_traintest.csv")
# Validar número de colunas
# Validar e tratar tipo das colunas (principalmente datas através da função pd.to_datetime)
tb_hotel_cleaned = tb_hotel[['is_cancelled',
                              'hotel',
                              'stays_in_weekend_nights',
                              'stays_in_week_nights',
                              'adults',
                              'children',
                              'babies',
                              #'meal',
                              #'country',
                              #'market_segment',
                              'is_repeated_guest',
                              'previous_cancellations',
                              'previous_bookings_not_canceled',
                              #'reserved_room_type',
                              'booking_changes',
                              #'deposit_type',
                              'days_in_waiting_list',
                              #'customer_type',
                              'adr',
                              'required_car_parking_spaces',
                              'total_of_special_requests'
                              #'reservation_status_date',
                              #'arrival_date'
                              ]]
tb_hotel_cleaned = tb_hotel_cleaned.dropna()

In [27]:
#transformação de variável input 'hotel' em numérico
tb_hotel_cleaned = pd.concat([pd.get_dummies(tb_hotel_cleaned['hotel'],dtype=int,drop_first = True),tb_hotel_cleaned],axis=1)
tb_hotel_cleaned = tb_hotel_cleaned.drop('hotel', axis=1)

In [28]:
tb_hotel_cleaned.head()

,Resort Hotel,is_cancelled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,0,0,2,0.0,0,0,0,0,3,0,0.0,0,0
1,1,0,0,0,2,0.0,0,0,0,0,4,0,0.0,0,0
2,1,0,0,1,1,0.0,0,0,0,0,0,0,75.0,0,0
3,1,0,0,1,1,0.0,0,0,0,0,0,0,75.0,0,0
4,1,0,0,2,2,0.0,0,0,0,0,0,0,98.0,0,1


In [29]:
# separando em variáveis X e Y
tb_hotel_cleaned_Y = tb_hotel_cleaned['is_cancelled']
tb_hotel_cleaned_X = tb_hotel_cleaned.drop(['is_cancelled'], axis=1)

# EDA (Análise Exploratória de Dados)

#### Avaliação de problemas nos dados (missing value e informações incorretas)

In [ ]:
tb_hotel.describe()

In [ ]:
tb_hotel.info()

In [ ]:
tb_hotel.head()

In [ ]:
sns.histplot(data=tb_hotel[['hotel','is_cancelled']],x='is_cancelled', hue='hotel', multiple="dodge")

In [ ]:
# dúvida
# Como faço pra plotar sem esses números do meio (precisa transformar em boolean)

In [ ]:
tb_city_resort = tb_hotel[['is_cancelled','hotel','id_booking']].groupby(by=['hotel', 'is_cancelled']).count()
print(tb_city_resort)

In [ ]:
tb_city_resort = tb_city_resort.reset_index()

In [ ]:
tb_city_resort['sum_bookings'] = tb_city_resort.groupby(by=['hotel']).transform(sum)['id_booking']
tb_city_resort['%cancelled'] = tb_city_resort['id_booking'] / tb_city_resort['sum_bookings']
print(tb_city_resort)

In [ ]:
tb_hotel.corr()

# Modelo Baseline

In [ ]:
# Variáveis:
# hotel -- criar boolean

In [ ]:
# dúvida
# todas as variáveis deram uma correlação meio baixa, é isso mesmo?

#### Divisão do dataset em train e test para evitar leakage

In [76]:
X_train, X_test, y_train, y_test = train_test_split(tb_hotel_cleaned_X, tb_hotel_cleaned_Y, test_size = 0.2)

In [77]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90724 entries, 93860 to 111687
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Resort Hotel                    90724 non-null  int32  
 1   stays_in_weekend_nights         90724 non-null  int64  
 2   stays_in_week_nights            90724 non-null  int64  
 3   adults                          90724 non-null  int64  
 4   children                        90724 non-null  float64
 5   babies                          90724 non-null  int64  
 6   is_repeated_guest               90724 non-null  int64  
 7   previous_cancellations          90724 non-null  int64  
 8   previous_bookings_not_canceled  90724 non-null  int64  
 9   booking_changes                 90724 non-null  int64  
 10  days_in_waiting_list            90724 non-null  int64  
 11  adr                             90724 non-null  float64
 12  required_car_parking_spaces

#### PCA evitará não convergência de regressão logística

In [78]:
norm = StandardScaler().fit(X_train)
X_train_norm = norm.transform(X_train)
pca_t = PCA()
pca_t.fit(X_train_norm)
X_train_pca = pca_t.transform(X_train_norm)

In [79]:
X_test_norm = norm.transform(X_test)
X_test_pca = pca_t.transform(X_test_norm)

#### Regressão logística

In [80]:
# falta evitar leakage
logistic = LogisticRegression()
logistic.fit(X=X_train_pca, y=y_train)

LogisticRegression()

In [83]:
tb_predict_log = pd.DataFrame({'y_real': y_test, 'y_predict': logistic.predict(X_test_pca)})

In [84]:
tb_predict_log.head()

,y_real,y_predict
46384,0,0
103692,0,1
40443,0,1
8961,1,0
29989,0,0


In [86]:
print(f1_score(y_true = tb_predict_log['y_real'],
             y_pred = tb_predict_log['y_predict']))

0.5387636363636364


#### 1-NN (kNN com n-neighbors = 1)

In [89]:
knn_fit = KNeighborsClassifier(n_neighbors=1)
knn_fit.fit(X_train_pca, y_train)

KNeighborsClassifier(n_neighbors=1)

In [90]:
tb_predict_KNN = pd.DataFrame({'y_real': y_test, 'y_predict': knn_fit.predict(X_test_pca)})

In [92]:
tb_predict_KNN.head()

,y_real,y_predict
46384,0,0
103692,0,1
40443,0,0
8961,1,0
29989,0,0


In [93]:
print(f1_score(y_true = tb_predict_KNN['y_real'],
             y_pred = tb_predict_KNN['y_predict']))

0.6609040158472897


#### Teste do sample

In [109]:
sample = pd.read_csv("tb_hotel_feat_valid.csv")
logistic_output = pd.DataFrame()
KNN_output = pd.DataFrame()

In [110]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5981 entries, 0 to 5980
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           5981 non-null   object 
 1   lead_time                       5981 non-null   int64  
 2   stays_in_weekend_nights         5981 non-null   int64  
 3   stays_in_week_nights            5981 non-null   int64  
 4   adults                          5981 non-null   int64  
 5   children                        5980 non-null   float64
 6   babies                          5981 non-null   int64  
 7   meal                            5981 non-null   object 
 8   country                         5951 non-null   object 
 9   market_segment                  5981 non-null   object 
 10  distribution_channel            5981 non-null   object 
 11  is_repeated_guest               5981 non-null   int64  
 12  previous_cancellations          59

In [111]:
sample = sample[['hotel',
                              'stays_in_weekend_nights',
                              'stays_in_week_nights',
                              'adults',
                              'children',
                              'babies',
                              #'meal',
                              #'country',
                              #'market_segment',
                              'is_repeated_guest',
                              'previous_cancellations',
                              'previous_bookings_not_canceled',
                              #'reserved_room_type',
                              'booking_changes',
                              #'deposit_type',
                              'days_in_waiting_list',
                              #'customer_type',
                              'adr',
                              'required_car_parking_spaces',
                              'total_of_special_requests',
                 'id_booking'
                              #'reservation_status_date',
                              #'arrival_date'
                              ]]
sample = sample.dropna()

In [112]:
logistic_output['id_booking'] = sample['id_booking'].copy()
KNN_output['id_booking'] = sample['id_booking'].copy()

In [113]:
sample_X = sample.drop(['id_booking'], axis=1)
sample_X = pd.concat([pd.get_dummies(sample_X['hotel'],dtype=int,drop_first = True),sample_X],axis=1)
sample_X = sample_X.drop('hotel', axis=1)

In [114]:
sample_X.head()

,Resort Hotel,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,2,5,2,0.0,0,0,0,0,0,0,82.88,0,2
1,1,1,0,2,0.0,0,0,0,0,0,0,97.00,0,0
2,1,1,3,2,2.0,0,0,0,0,0,0,210.00,1,0
3,1,1,1,2,0.0,0,0,0,0,0,0,87.30,1,1
4,1,1,5,2,0.0,0,0,0,0,0,0,117.80,1,1


In [115]:
sample_X_norm = norm.transform(sample_X)
sample_X_pca = pca_t.transform(sample_X_norm)

In [118]:
KNN_output['is_cancelled'] = pd.DataFrame(knn_fit.predict(sample_X_pca))
logistic_output['is_cancelled'] = pd.DataFrame(logistic.predict(sample_X_pca))

In [119]:
KNN_output.head()

,id_booking,is_cancelled
0,47,0.0
1,132,1.0
2,164,0.0
3,171,0.0
4,178,0.0


In [120]:
logistic_output.head()

,id_booking,is_cancelled
0,47,0.0
1,132,0.0
2,164,0.0
3,171,0.0
4,178,0.0


In [124]:
logistic_output.to_csv('out_log.csv',index=False)

In [125]:
KNN_output.to_csv('out_knn.csv',index=False)